> # **요구사항 1**

In [58]:
import os # 파일 경로, 파이토치 라이브러리
import pandas as pd # 데이터분석 라이브러리
import torch # tensor 관련 라이브리리
from torch.utils.data import Dataset, DataLoader, random_split # 데이터셋, 랜덤 스플릿 기능이 들어간 라이브러리를


class TitanicDataset(Dataset):
  def __init__(self, X, y):
    self.X = torch.FloatTensor(X) # input값들을 32bit 실수형 텐서로 변환
    self.y = torch.LongTensor(y) # target값들을 64bit 정수형 텐서로 변환

  def __len__(self): # dataset의 길이 반환
    return len(self.X)

  def __getitem__(self, idx): # 데이터를 인덱싱하여 반환 
    feature = self.X[idx]
    target = self.y[idx]
    return {'input': feature, 'target': target}

  def __str__(self): # print가 사용될 때 dataset를 특정 문장에 맞게 출력함
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape
    )
    return str

In [138]:
# TitanicDatset Test
feature = torch.rand( size=(10, 2) )
target = torch.randint( low=0, high=1, size=(10,) )
titanicDataset = TitanicDataset(feature, target)
print( f"first data : { titanicDataset[0] }" )
print( f"len : { len(titanicDataset) }" )
print( titanicDataset )

first data : {'input': tensor([0.5002, 0.9984]), 'target': tensor(0)}
len : 10
Data Size: 10, Input Shape: torch.Size([10, 2]), Target Shape: torch.Size([10])


In [60]:
class TitanicTestDataset(Dataset):
  def __init__(self, X): # dataset 생성 시 input값 하나만 받아 32bit 실수형 텐서로 변환 후 저장.
    self.X = torch.FloatTensor(X) 

  def __len__(self): # dataset의 길이 반환
    return len(self.X)

  def __getitem__(self, idx): # 인덱스 값이 들어오면 dataset에서 인덱싱 한 값 반환
    feature = self.X[idx]
    return {'input': feature}

  def __str__(self): # print가 들어올 때 설정한 문장을 반환함
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape
    )
    return str

In [139]:
# TitanicTestDataset Test
titanicTestDataset = TitanicTestDataset(torch.randn(size=(10, 10)))
print( "len : {0}".format( len( titanicTestDataset ) ) )
print("first data : {0}".format( titanicTestDataset[0]))
print(titanicTestDataset)

len : 10
first data : {'input': tensor([ 0.2317, -1.3007, -0.8115,  0.4755,  0.5561,  0.9156, -0.7379, -1.0000,
        -1.4148,  2.1031])}
Data Size: 10, Input Shape: torch.Size([10, 10])


In [62]:
def get_preprocessed_dataset():
    #CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__))

    #train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    #test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    train_data_path = os.path.join(os.path.pardir, os.path.pardir, "link_dl", "_03_your_code", "train.csv")
    test_data_path = os.path.join(os.path.pardir, os.path.pardir, "link_dl", "_03_your_code", "test.csv")

    train_df = pd.read_csv(train_data_path) # train.csv 불러오기
    test_df = pd.read_csv(test_data_path) # test.csv 불러오기

    all_df = pd.concat([train_df, test_df], sort=False) # train_df + test_df / 전처리(preprocessed)과정에서 더 일관된 데이터 학습을 위해 두 데이터를 합침.
    
    all_df = get_preprocessed_dataset_1(all_df)

    all_df = get_preprocessed_dataset_2(all_df)

    all_df = get_preprocessed_dataset_3(all_df)

    all_df = get_preprocessed_dataset_4(all_df)

    all_df = get_preprocessed_dataset_5(all_df)

    all_df = get_preprocessed_dataset_6(all_df)

    #all_df에서 "Survived" feature가 NaN이 아닌 행 중에서 Survived열 데이터를 버린 후 데이터 프레임 형태로 반환
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True) 
    train_y = train_df["Survived"]
    #all_df 중 "Survived" feature가 Nan인 행 중에 Survived열 데이터를 버린 후 데이터 프레임 형태로 반환함.
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    #전처리한 데이터 셋을 합쳐서 반환함.
    dataset = TitanicDataset(train_X.values, train_y.values)
    #print(dataset)
    # 전처리된 데이터셋을 랜덤으로 분할해 train과 validation dataset에 담아줌.
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset


In [63]:
def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    # dataset 내의 "Pclass"를 기준으로 그룹화해 "Pclass"별 "Fare"값 평균을 구해 데이터 프레임으로 반환함. 
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index() 
    Fare_mean.columns = ["Pclass", "Fare_mean"] # Fare_mean의 feature 이름을 ,Pclass, Fare_mean으로 변경.
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left") # Fare_mean을 Pclass 기준으로 all_df 오른쪽에 합치기
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"] # "Fare" feature 중에 결측치 값을 Fare_mean으로 채워 결측치 없에기

    return all_df


In [64]:
# get_preprocessed_dataset_1 테스트
train_data_path = os.path.join(os.path.pardir, os.path.pardir, "link_dl", "_03_your_code", "train.csv")
test_data_path = os.path.join(os.path.pardir, os.path.pardir, "link_dl", "_03_your_code", "test.csv")
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)
Test_all_df = pd.concat([train_data, test_data], sort=False)

print("[before]")
print("-"*100)
print(Test_all_df["Fare"].head(10))
print(" ")

preprocessed_train_data = get_preprocessed_dataset_1(Test_all_df)

print("[after]")
print("-"*100)
print(preprocessed_train_data["Fare"].head(10))

[before]
----------------------------------------------------------------------------------------------------
0     7.2500
1    71.2833
2     7.9250
3    53.1000
4     8.0500
5     8.4583
6    51.8625
7    21.0750
8    11.1333
9    30.0708
Name: Fare, dtype: float64
 
[after]
----------------------------------------------------------------------------------------------------
0     7.2500
1    71.2833
2     7.9250
3    53.1000
4     8.0500
5     8.4583
6    51.8625
7    21.0750
8    11.1333
9    30.0708
Name: Fare, dtype: float64


In [65]:
def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True) # name feature에 있는 값들을 , .을 기준으로 2번 나눔. 또한 각각의 열을 확장시켜서 추가함.
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip() # family_name feature의 공백 제거
    name_df["honorific"] = name_df["honorific"].str.strip() # honorific feature의 공백 제거
    name_df["name"] = name_df["name"].str.strip() # name feature의 공백 제거
    all_df = pd.concat([all_df, name_df], axis=1) # family_name, honorific, name 데이터 프레임을 모두 합침.

    return all_df

In [66]:
# get preprocessed_dataset_2 test
print("[before] ")
print("-"*100)
print(preprocessed_train_data["Name"].head(5))
print(" ")
preprocessed_train_data2 = get_preprocessed_dataset_2(preprocessed_train_data)
print("[after]")
print("-"*100)
print(preprocessed_train_data2["name"].head(5))

[before] 
----------------------------------------------------------------------------------------------------
0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                             Allen, Mr. William Henry
Name: Name, dtype: object
 
[after]
----------------------------------------------------------------------------------------------------
0                              Owen Harris
1    John Bradley (Florence Briggs Thayer)
2                                    Laina
3            Jacques Heath (Lily May Peel)
4                            William Henry
Name: name, dtype: object


In [67]:
def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    # all_df의 honorific, Age를 honorific을 기준으로 그룹화(Mr끼리, Mis끼리, Mrs끼리) 후 반올린한 평균값을 가지는 데이터 프레임 반환
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index() 
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ] # honorific_age_mean의 열 이름을 변경
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left") # "honorific"을 기준으로 all_df 우측에 honorific_age_mean 합치기
    # loc로 "Age" feature의 값들에 대해 Null이라면 "Age" 값들을 "honorific_age_mean"값으로 대체
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"] 
    all_df = all_df.drop(["honorific_age_mean"], axis=1) # "honorific_age_mean"열 제거

    return all_df


In [68]:
print("[before]")
print("-"*100)
print(preprocessed_train_data2["Age"].head(10))
print(" ")

preprocessed_train_data3 = get_preprocessed_dataset_3(preprocessed_train_data2)

print("[after]")
print("-"*100)
print(preprocessed_train_data3["Age"].head(10))

[before]
----------------------------------------------------------------------------------------------------
0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64
 
[after]
----------------------------------------------------------------------------------------------------
0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5    29.0
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64


In [69]:
def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"] # "family_num" column을 새로 생성, "Parch" 값과 "SibSp"값을 더해 넣음

    # 혼자탑승(alone) 컬럼 새롭게 추가
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1 # "family_num" column값이 0이면 새로 생성한 "alone" column에 1 넣음.
    all_df["alone"].fillna(0, inplace=True) # "alone" column의 NaN인 값에 0을 대입하고 즉각 데이터프레임에 반영함.

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df

In [70]:
print("[before]")
print("-"*100)
print(preprocessed_train_data3.head(3))
print("")

print("[after]")
print("-"*100)
preprocessed_train_data4 = get_preprocessed_dataset_4(preprocessed_train_data3)
print(preprocessed_train_data4.head(3))

[before]
----------------------------------------------------------------------------------------------------
   PassengerId  Survived  Pclass  \
0            1       0.0       3   
1            2       1.0       1   
2            3       1.0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   

   Parch            Ticket     Fare Cabin Embarked  Fare_mean family_name  \
0      0         A/5 21171   7.2500   NaN        S  13.302889      Braund   
1      0          PC 17599  71.2833   C85        C  87.508992     Cumings   
2      0  STON/O2. 3101282   7.9250   NaN        S  13.302889   Heikkinen   

  honorific                                   name  
0        Mr                            Owen Harris  
1       Mrs  John Bra

C:\Users\User\AppData\Local\Temp\ipykernel_16164\3311476485.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True) # "alone" column의 NaN인 값에 0을 대입하고 즉각 데이터프레임에 반영함.


   Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked  Fare_mean  \
0       0.0       3    male  22.0      1      0   7.2500        S  13.302889   
1       1.0       1  female  38.0      1      0  71.2833        C  87.508992   
2       1.0       3  female  26.0      0      0   7.9250        S  13.302889   

  honorific  family_num  alone  
0        Mr           1    0.0  
1       Mrs           1    0.0  
2      Miss           0    1.0  


In [71]:
def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other" # all_df의 "honorific" column값이 "Mr", "Miss", "Mrs", "Master"가 아닌 경우 "honorific"에 "other" 대입
    all_df["Embarked"].fillna("missing", inplace=True) # "Embarked" column 값이 NAN일 경우 "missing" 대입. 데이터프레임에 즉각적으로 반영함

    return all_df


In [140]:
# get_preprocessed_dataset_5 Test
print("[before]")
print("-"*100)
print(preprocessed_train_data4[["honorific", "Embarked"]].tail(20))
print()

print("[after]")
print("-"*100)
preprocessed_train_data5 = get_preprocessed_dataset_5(preprocessed_train_data4)
print(preprocessed_train_data5[["honorific", "Embarked"]].tail(20))

[before]
----------------------------------------------------------------------------------------------------
      honorific  Embarked
1289          2         2
1290          2         1
1291          1         2
1292          2         2
1293          1         0
1294          2         2
1295          2         0
1296          2         0
1297          2         2
1298          2         0
1299          1         1
1300          1         2
1301          1         1
1302          3         1
1303          1         2
1304          2         2
1305          4         0
1306          2         2
1307          2         2
1308          0         0

[after]
----------------------------------------------------------------------------------------------------


C:\Users\User\AppData\Local\Temp\ipykernel_16164\2126945265.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'other' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  all_df.loc[
C:\Users\User\AppData\Local\Temp\ipykernel_16164\2126945265.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["Embarked"].fillna("missing", inplace=True) # "Embarked" column 값이 NAN일 경우 "missing" 대입. 데이터프레임에 즉각적으로 반영함


     honorific  Embarked
1289     other         2
1290     other         1
1291     other         2
1292     other         2
1293     other         0
1294     other         2
1295     other         0
1296     other         0
1297     other         2
1298     other         0
1299     other         1
1300     other         2
1301     other         1
1302     other         1
1303     other         2
1304     other         2
1305     other         0
1306     other         2
1307     other         2
1308     other         0


In [73]:
def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    category_features = all_df.columns[all_df.dtypes == "object"] # all_df의 데이터들이 "object"타입인 열을 반환함.
    from sklearn.preprocessing import LabelEncoder # 카테고리 변수를 수치화해주는 LabelEncoder를 import함
    for category_feature in category_features:
        le = LabelEncoder() #LabelEncoder의 인스턴스 생성
        if all_df[category_feature].dtypes == "object":
          le = le.fit(all_df[category_feature]) # LabelEncoder가 카테고리 변수에 대해 학습.
            # "category_feature"에 해당하는 all_df의 column의 값을 학습한 데이터를 토대로 값들을 수치화 해 대입함.
          all_df[category_feature] = le.transform(all_df[category_feature]) 

    return all_df

In [142]:
# get_preprocessed_dataset_6 Test
category_feature = preprocessed_train_data5.columns[preprocessed_train_data5.dtypes == "object"]


print("[before]")
print("-"*100)
print(preprocessed_train_data5[category_feature])
print()

print("[after]")
print("-"*100)
preprocessed_train_data6 = get_preprocessed_dataset_6(preprocessed_train_data5)
print(preprocessed_train_data6[["Sex", "Embarked", "honorific"]])


[before]
----------------------------------------------------------------------------------------------------
     honorific
0        other
1        other
2        other
3        other
4        other
...        ...
1304     other
1305     other
1306     other
1307     other
1308     other

[1309 rows x 1 columns]

[after]
----------------------------------------------------------------------------------------------------
      Sex  Embarked  honorific
0       1         2          0
1       0         0          0
2       0         2          0
3       0         2          0
4       1         2          0
...   ...       ...        ...
1304    1         2          0
1305    0         0          0
1306    1         2          0
1307    1         2          0
1308    1         0          0

[1309 rows x 3 columns]


In [144]:
from torch import nn, optim # 최적화, 뉴럴 링크 라이브러리 불러오기
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__() # 부모 클래스인 nn.Module의 생성자를 사용
        self.model= nn.Sequential( # hidden layer 1개와 input, output layer 한개로 구성된 모델 생성. 활성함수로 ReLU 선택
            nn.Linear(n_input, 30), # input = 30 / output = 30
            nn.ReLU(),
            nn.Linear(30, 30), # input = 30 / output = 30
            nn.ReLU(),
            nn.Linear(30, n_output),
         )
    def forward(self, x):
        x = self.model(x) # 모델 반환
        return x

In [145]:
# Mymodel Test
input = torch.ones(size = (10,))
print(input)
model = MyModel(10, 1)
model.forward(input)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


tensor([-0.1597], grad_fn=<ViewBackward0>)

In [77]:
def test(test_data_loader):
  print("[TEST]")
  batch = next(iter(test_data_loader)) # test_data_loader에 설정된 batch_size만큼 데이터를 반복적으로 batch에 불러옴
  print("{0}".format(batch['input'].shape))
  my_model = MyModel(n_input=11, n_output=2) # 11개의 입력과 2개의 출력, 하나의 히든 레이어로 이루어진 MyModel을 생성.
  output_batch = my_model(batch['input']) # batch에 있는 값을 넣어 학습시킨 결과를 output_batch에 입력
  prediction_batch = torch.argmax(output_batch, dim=1) # ㅐoutput_batch의 1차원 값 중 가장 높은 값의 인덱스를 prediction_batch에 저장
  for idx, prediction in enumerate(prediction_batch, start=892): 
      print(idx, prediction.item()) #인덱스 값들을 출력

In [66]:
if __name__ == "__main__":
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset() # 데이터를 전처리 한 후 train, validation, test용으로 나눠서 저장

  print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
    len(train_dataset), len(validation_dataset), len(test_dataset)
  ))
  print("#" * 50, 1)

  for idx, sample in enumerate(train_dataset):
    print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

  print("#" * 50, 2)

  train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))
  print(train_dataset[0])
  print("[TRAIN]")
  for idx, batch in enumerate(train_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("[VALIDATION]")
  for idx, batch in enumerate(validation_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("#" * 50, 3)

  test(test_data_loader)

C:\Users\User\AppData\Local\Temp\ipykernel_6664\3311476485.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True) # "alone" column의 NaN인 값에 0을 대입하고 즉각 데이터프레임에 반영함.
C:\Users\User\AppData\Local\Temp\ipykernel_6664\2126945265.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting val

Data Size: 418, Input Shape: torch.Size([418, 11])
train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 3.0000,  0.0000, 22.0000,  0.0000,  0.0000,  7.7875,  1.0000, 13.3029,
         1.0000,  0.0000,  1.0000]): 1
1 - tensor([ 1.0000,  1.0000, 50.0000,  1.0000,  0.0000, 55.9000,  2.0000, 87.5090,
         2.0000,  1.0000,  0.0000]): 0
2 - tensor([ 2.0000,  0.0000, 57.0000,  0.0000,  0.0000, 10.5000,  2.0000, 21.1792,
         3.0000,  0.0000,  1.0000]): 0
3 - tensor([ 3.0000,  1.0000, 28.0000,  0.0000,  0.0000,  7.7958,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
4 - tensor([ 3.0000,  0.0000, 18.0000,  0.0000,  0.0000,  6.7500,  1.0000, 13.3029,
         1.0000,  0.0000,  1.0000]): 0
5 - tensor([ 3.0000,  1.0000, 16.0000,  0.0000,  0.0000,  8.0500,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 1
6 - tensor([ 2.0000,  0.0000, 19.0000,  0.0000,  0.0000, 26.0000,  2.0000, 21.1792,
     

> # **요구사항 2**

In [136]:
import os
import torch
from torch import nn, optim # 뉴럴 네트워크, optimizer를 사용할 수 있게 하는 라이브러리
from torch.utils.data import random_split, DataLoader # dataloader 사용 라이브러리
from datetime import datetime # 시간값을 반환하는 라이브러리
import wandb # wandb 사용 라이브러리
import argparse # 커멘드 라인에서 프로그램에 인자를 전달하는 라이브리리
import pandas # 데이터 처리를 원활하게 해주는 라이브리리


from pathlib import Path

BASE_PATH = str( os.path.join(os.path.pardir, os.path.pardir, "link_dl") )
print(BASE_PATH, "!!!!!!!")

import sys
sys.path.append(BASE_PATH)


def get_data():
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
    
  print(len(train_dataset), len(validation_dataset))
  train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

  return train_data_loader, validation_data_loader, test_data_loader

def Swish(x): # sigmoid에 입력값은 곱한 활성함수. RELU보다 좋은 성능을 내고 있다는 결과가 있음
    return x * torch.sigmoid(x)

class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    self.model = nn.Sequential( # 입력 층, 히든 레이어층 3개, 출력 층으로 구성
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
      nn.LeakyReLU(),
      nn.BatchNorm1d(wandb.config.n_hidden_unit_list[0]),
      #nn.Dropout(p=0.2), 사용하면 early stop이 빨리 일어남.
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      nn.LeakyReLU(),
      nn.BatchNorm1d(wandb.config.n_hidden_unit_list[1]), # 데이터를 정규화시킴
      nn.Linear(wandb.config.n_hidden_unit_list[2], wandb.config.n_hidden_unit_list[2]),
      nn.LeakyReLU(),
      nn.BatchNorm1d(wandb.config.n_hidden_unit_list[2]),
        
      nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x

def get_model_and_optimizer(): # 사전에 설정한 model과 optimizer 반환
  my_model = MyModel(n_input=11, n_output=1)
  optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate, weight_decay=0.01)

  return my_model, optimizer


def training_loop(model, optimizer, train_data_loader, validation_data_loader):
  n_epochs = wandb.config.epochs
  loss_fn = nn.BCEWithLogitsLoss() # binary classfication에 사용되는 loss함수. 결과값에 sigmoid를 적용해야하는 BCE와 달리 sigmoid가 내부적으로 적용된다.
  next_print_epoch = 100
  count = 0
  min_validation = None
  #print(model.state_dict())
  #ccc = 1
  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:

      output_train = model(train_batch['input']).squeeze() # 모델을 통해 나온 결과값을 저장(target의 shape와 맞추기 위해 squeeze()함)
      target = train_batch['target'].float() # BCEWithLogitsLoss는 실수값을 가지고 비교함.

      loss = loss_fn(output_train, target) # 모델을 통해 나온 결과값과 정답을 비교
      loss_train += loss.item() # loss값을 출력하기 위한 틀린 횟수 저장
      num_trains += 1 # 학습한 횟수 기록

      optimizer.zero_grad() # gradient 값 초기화
      loss.backward() # backpropagation을 진행하면서 gradient 값 측정
      optimizer.step() # gradient값을 가지고 파라미터 업데이트

    loss_validation = 0.0
    num_validations = 0
    
    
    with torch.no_grad(): # 아래 실행과정은 기록하지 않음
      for validation_batch in validation_data_loader:
          
        output_validation = model(validation_batch["input"]).squeeze() # target tensor와 shape를 맞추기 위해 squeeze() 적용
        target = validation_batch['target'].float()
        
        loss = loss_fn(output_validation, target)# validation 결과값을 가지고 loss값 생성
        
        #early stopping code
        if min_validation is None: # 검증 초반의 loss값으로 min_validation을 초기화함
            min_validation = loss
        elif min_validation > loss - 0.001: # loss가 이전 loss - delta값보다 적으면, 즉 loss가 이전보다 감소했다면 count값 초기화
            min_validation = loss
            count = 0
        else: # loss가 감소되지 않으면 count값 추가
            count += 1

        if count >= 10: # 10번 이상 loss가 감소되지 않으면 학습 종료 후 모델 반환
            print("early stop!!!")
            return model
        loss_validation += loss.item() # 한 batch에 대해 검증이 완료된 후의 loss 결과값을 loss_validation에 넣기.
        num_validations += 1 # 검증 횟수 기록하기

    wandb.log({
      "Epoch": epoch,
      "Training loss": loss_train / num_trains,
      "Validation loss": loss_validation / num_validations
    })
    
    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}"
      )
      next_print_epoch += 100
  # torch.save(model.state_dict(), 'model_weight.pth') #학습한 모델의 파라미터를 local의 model_weight.pth에 기록
  return model #학습한 모델을 반환 

def main(args):
  current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S') # 현재 시각을 반환하는 datatime.now() 함수

  config = {
    'epochs': args.epochs,
    'batch_size': args.batch_size,
    'learning_rate': 1e-3,
    'n_hidden_unit_list': [20, 20, 20],
  }

  wandb.init(
    mode="online" if args.wandb else "disabled",
    project="my_model_training",
    notes="My first wandb experiment",
    tags=["my_model", "titanic_disaster"],
    name=current_time_str,
    config=config
  )
  print(args)
  print(wandb.config)

  train_data_loader, validation_data_loader, test_data_loader = get_data() # get_data()를 통해 모델에 사용할 데이터를 전처리하여 반환

  linear_model, optimizer = get_model_and_optimizer() # 사용할 모델, optimizer 반환

  print("#" * 50, 1)

  after_model = training_loop(
    model=linear_model,
    optimizer=optimizer,
    train_data_loader=train_data_loader,
    validation_data_loader=validation_data_loader
  )

  wandb.finish()

  return after_model, test_data_loader

# https://docs.wandb.ai/guides/track/config
if __name__ == "__main__":
  parser = argparse.ArgumentParser()

  parser.add_argument(
    "--wandb", action=argparse.BooleanOptionalAction, default=True, help="True or False"
  )

  parser.add_argument(
    "-b", "--batch_size", type=int, default=512, help="Batch size (int, default: 512)"
  )

  parser.add_argument(
    "-e", "--epochs", type=int, default=1_500, help="Number of training epochs (int, default:1_000)"
  )

  args = parser.parse_args(args=[]) # jupyter notebook에서 argparse를 사용하기 위해 args = default로 설정함.

  after_model, test_data_loader = main(args)



..\..\link_dl !!!!!!!


Namespace(wandb=True, batch_size=512, epochs=1500)
{'epochs': 1500, 'batch_size': 512, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20, 20]}


C:\Users\User\AppData\Local\Temp\ipykernel_16164\3311476485.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True) # "alone" column의 NaN인 값에 0을 대입하고 즉각 데이터프레임에 반영함.
C:\Users\User\AppData\Local\Temp\ipykernel_16164\2126945265.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting v

713 178
################################################## 1
Epoch 100, Training loss 0.6385, Validation loss 0.6250
Epoch 200, Training loss 0.6138, Validation loss 0.6071
Epoch 300, Training loss 0.5878, Validation loss 0.5982
Epoch 400, Training loss 0.5699, Validation loss 0.5955
Epoch 500, Training loss 0.5617, Validation loss 0.5903
Epoch 600, Training loss 0.5592, Validation loss 0.5846
Epoch 700, Training loss 0.5513, Validation loss 0.5793
Epoch 800, Training loss 0.5516, Validation loss 0.5739
Epoch 900, Training loss 0.5446, Validation loss 0.5697
Epoch 1000, Training loss 0.5455, Validation loss 0.5671
Epoch 1100, Training loss 0.5392, Validation loss 0.5616
Epoch 1200, Training loss 0.5284, Validation loss 0.5585
Epoch 1300, Training loss 0.5471, Validation loss 0.5531
Epoch 1400, Training loss 0.5149, Validation loss 0.5495
Epoch 1500, Training loss 0.5154, Validation loss 0.5474


Epoch,▁▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇███
Training loss,█▅▄▅▅▄▄▄▅▄▄▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▃▂▁▁▁▂▁▁▁▁▁▂
Validation loss,█▇▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
Epoch,1500
Training loss,0.51539
Validation loss,0.54736


> # **요구사항 3**

In [137]:
def test(model, test_data_loader):
    results = [] # 테스트 결과값을 저장할 변수
    
    batch = next(iter(test_data_loader)) # 설정한 batch_size만큼 데이터를 계속 불러옴
    with torch.no_grad():
        output_batch = model(batch['input'])
        prediction_batch = (output_batch >= 0.5).int() # 임계값을 이용해 결과값들을 0 또는 1로 바꿔줌
        for idx, prediction in enumerate(prediction_batch, start=892): 
            print(idx, prediction.item()) #인덱스 값들을 출력
            results.append({"PassengerId": idx, "Survived": prediction.item()}) # 테스트 결과에 추가함
            
    df = pd.DataFrame(results) # 테스트 결과값을 데이터프레임 형태로 만듬

    df.to_csv('submission.csv', index=False) # csv파일 형태로 인덱스 없이 저장
    
    print("test complete and Results saved to submission.csv")


test(after_model, test_data_loader=test_data_loader)

892 0
893 0
894 0
895 0
896 0
897 0
898 0
899 0
900 0
901 0
902 0
903 0
904 1
905 0
906 1
907 0
908 0
909 0
910 0
911 0
912 0
913 1
914 0
915 1
916 0
917 0
918 1
919 0
920 0
921 0
922 0
923 0
924 0
925 0
926 0
927 0
928 0
929 0
930 0
931 0
932 0
933 0
934 0
935 0
936 1
937 0
938 0
939 0
940 1
941 0
942 1
943 0
944 0
945 1
946 0
947 0
948 0
949 0
950 0
951 0
952 0
953 0
954 0
955 0
956 1
957 0
958 0
959 0
960 0
961 0
962 0
963 0
964 0
965 0
966 0
967 0
968 0
969 0
970 0
971 0
972 1
973 0
974 0
975 0
976 0
977 0
978 0
979 0
980 0
981 1
982 0
983 0
984 1
985 0
986 0
987 0
988 0
989 0
990 0
991 0
992 0
993 0
994 0
995 0
996 0
997 0
998 0
999 0
1000 0
1001 0
1002 0
1003 0
1004 0
1005 0
1006 0
1007 0
1008 0
1009 1
1010 1
1011 0
1012 1
1013 0
1014 1
1015 0
1016 0
1017 0
1018 0
1019 0
1020 0
1021 0
1022 0
1023 0
1024 1
1025 0
1026 0
1027 0
1028 0
1029 0
1030 0
1031 0
1032 0
1033 1
1034 0
1035 0
1036 0
1037 0
1038 1
1039 0
1040 0
1041 0
1042 1
1043 0
1044 0
1045 0
1046 0
1047 0
1048 1
1049 0
10

> # **요구사항 4**

![titanic_제출등수.png](https://github.com/minimiYakgwa/first-repository/blob/master/%EB%A7%88%EC%A7%80%EB%A7%89%ED%95%99%EC%8A%B5%EA%B2%B0%EA%B3%BC.png?raw=true)



> # **느낀점**

+ data preprocessed 코드를 해석해보면서 tensor에 대한 중요성을 크게 느꼈습니다. 
+ pandas에 대해 익숙했다면 코드 분석이 좀 더 빠르게 진행되었을 것 같았고, 결측치를 메꾸는 과정에서 관련데이터를 사용한다고 해도 학습에 영향을 줄텐데 괜찮은지 궁금하게 되었습니다.
+ ReLU, ELU보다 Leaky Relu를 사용했을 경우 가장 좋은 loss 그래프를 얻을 수 있었고, trainning loss 그래프가 다른 활성함수에 비해 안정적이라는 것을 알 수 있었습니다.
+ binary classification에선 BCEWithLogitsLoss 함수를 많이 사용한다고 해서 사용해보기도 하였음. 확실히 일반 BCE 함수를 사용하면 마지막에 sigmoid를 적용해야하는 번거로움이 없어 편했습니다.
+ nn.Dropout() 방식도 사용해 봤는데, 빠르게 early stop이 걸리는 걸 알 수 있었습니다.
+ batch normalization()이라는 데이터 정규화 기법이란 것을 사용하니 너무 빠르게 가파르던 곡선이 안정적인 곡선으로 바뀌는 걸 알 수 있었습니다.
+ 학습이 잘 되지 않았음에도 early stop이 되는 느낌이 들어 delta값을 약간 조정하기도 했고, 이 과정에서 delta값을 어느정도 해야 될지 전혀 모르겠다고 생각했습니다.
+ 높은 결과가 한번 나온 적이 있었는데, 다시 한번 해봤을 때 결과가 안나왔습니다. 운좋게 학습이 잘되는 경우도 있는 듯 하다고 생각했습니다.
+ 여러 활성함수, 손실함수가 존재하는데, 모두 사용하는 상황이 다르다는 것을 알게 되었습니다. 함수가 모델에 미치는 영향을 몸소 체험할 수 있었습니다.
+ 코드를 수정해보면서 오류가 발생했을 땐 tensor의 형태를 살펴보는 동작을 먼저 했었고, 이러한 과정이 문제 해결에 상당한 도움이 되었다고 생각하게 되었습니다.
+ Layer 층을 늘리거나, 정규화를 거치거나, 파라미터를 일부 무시하거나 등 학습성능을 끌어올릴 여러 방법이 있다는 것을 알 수 있었지만, 막상 적용해보려고 할 때 어떨 때 적용해야 성능을 끌어올릴 수 있을지 막막했습니다. 이를 통해 경험이 중요하다는 교수님의 말씀을 잘 알 수 있었습니다. 